# Training SVM baseline on Feit-Thompson dataset
We train a SVM baseline on a set of heuristic features like context size, goal size, number of hypothesis in context and the smallest edit distance between a hypothesis and the context.

In [1]:
cd ..

/Users/prafulla/openai/gamepad_org


In [2]:
import argparse
import pickle
import torch
import numpy as np

from ml.fold_model import TacStModel
from ml.fold_train import TacStTrainer
# from ipdb import launch_ipdb_on_exception
# from ml.rewrite.solver import to_goalattn_dataset, run
from ml.rewrite.simprw import run_end2end
from ml.rewrite.dataset_prep import to_goalattn_dataset
from ml.tacst_prep import Dataset, TacStPt
from coq.tactics import TACTICS_EQUIV

## Loading dataset. 
Note that this is slightly smaller than the full dataset as the edit-distance calculation timed out on the biggest trees. 

In [4]:
with open("tacst_edit.pickle", "rb") as f:
    tacst_dataset, kern_tokens_to_idx, mid_tokens_to_idx = pickle.load(f)

In [5]:
print("Points Train={} Val={} Test={}".format(len(tacst_dataset.train), len(tacst_dataset.val), len(tacst_dataset.test)))

Points Train=61297 Val=7503 Test=7622


## Fitting SVM models
We note that the features look approximately poisson, while the log features look approximately gaussian, hence we train the SVM models on the log features. 

In [6]:
from sklearn import svm

def svm_models(dataset):
    for typ in ["mid_noimp", "kern"]:
        for targ in ["subtr_bin", "tac_bin"]:
            
            size_features = ['%s_concl_size' % typ, '%s_ctx_size' % typ]
            len_features = ['len_ctx']
            edit_dist_features = ['%s_str_dist' % typ]
            features = size_features + len_features  + edit_dist_features
            
            def _get_features(pt, features = features):
                return [getattr(pt, f) for f in features]
            
            def _get_targets(pt, targ = targ):
                return getattr(pt, targ)
            
            def get_xy(dataset):
                X = np.asarray([_get_features(pt) for (tactr_id,pt) in dataset])
                Y = np.asarray([_get_targets(pt) for (tactr_id,pt) in dataset])
                return X,Y
            
            x_t, y_t = get_xy(dataset.train)
            x_v, y_v = get_xy(dataset.val)
            x_s, y_s = get_xy(dataset.test)
            
            clf = svm.SVC()
            clf.fit(np.log(1+x_t), y_t)
            score_t = clf.score(np.log(1+x_t), y_t)
            score_v = clf.score(np.log(1+x_v), y_v)
            score_s = clf.score(np.log(1+x_s), y_s)
            print(typ, targ, "%.4f %.4f %.4f" % (score_t, score_v, score_s))

In [8]:
svm_models(tacst_dataset)

mid_noimp subtr_bin 0.5994 0.6008 0.5752
mid_noimp tac_bin 0.4933 0.5119 0.4945
kern subtr_bin 0.5975 0.6039 0.5737
kern tac_bin 0.4899 0.5045 0.4894
